In [9]:
## assigne api keys
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['HP_TOKEN'] = os.getenv('HF_TOKEN')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

groq_api_key = os.getenv("GROQ_API_KEY")

In [ ]:
## load documnets
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('data/Attention.pdf')
documents = loader.load()



## store vectores in vectorstoredb

## create retrieval

## invoke or retrive relevent info based on the questions

In [12]:
## splite docs
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = splitter.split_documents(documents=documents)

In [35]:
## embed docs
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
# doc_fun = lambda x: x.page_content
# docs = list(map(doc_fun, splits))
# vectors = embeddings.embed_documents(docs)


In [36]:
from langchain_community.vectorstores import FAISS

vectordb = FAISS.from_documents(splits, embeddings)

In [38]:
results = vectordb.similarity_search("What is transformers?")
print(f"{results[0].page_content}\n")


translation quality after being trained for as little as twelve hours on eight P100 GPUs.
2 Background
The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU
[16], ByteNet [18] and ConvS2S [9], all of which use convolutional neural networks as basic building
block, computing hidden representations in parallel for all input and output positions. In these models,
the number of operations required to relate signals from two arbitrary input or output positions grows
in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes
it more difficult to learn dependencies between distant positions [ 12]. In the Transformer this is
reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.



In [46]:
from langchain_groq import ChatGroq
model = 'llama3-8b-8192'
llm = ChatGroq(groq_api_key=groq_api_key, model=model)

In [47]:
from langchain_core.prompts import ChatPromptTemplate
## create a chat prompt
prompt = ChatPromptTemplate.from_template(
    """
    Answer the questions base on the provided context only.
    Pease provide the most accurate response based on the questions
    <context>
    {context}
    <context>
    Question:{input}
    """
)

In [48]:
## retriever
from langchain.chains import create_retrieval_chain
## combine documnets
from langchain.chains.combine_documents import create_stuff_documents_chain
## this is the document chain
document_chain = create_stuff_documents_chain(llm, prompt)
## create a retriever
retriever = vectordb.as_retriever()
## create retriever chain
retriever_chain = create_retrieval_chain(retriever, document_chain)

In [62]:

## create response or query result
user_query = "what is attention machenish?"
response = retriever_chain.invoke({"input": user_query})

In [64]:
print(response['input'])
print(response['context'][0].page_content)

what is attention machenish?
Attention Visualizations
Input-Input Layer5
It
is
in
this
spirit
that
a
majority
of
American
governments
have
passed
new
laws
since
2009
making
the
registration
or
voting
process
more
difficult
.
<EOS>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
It
is
in
this
spirit
that
a
majority
of
American
governments
have
passed
new
laws
since
2009
making
the
registration
or
voting
process
more
difficult
.
<EOS>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
Figure 3: An example of the attention mechanism following long-distance dependencies in the
encoder self-attention in layer 5 of 6. Many of the attention heads attend to a distant dependency of
the verb ‘making’, completing the phrase ‘making...more difficult’. Attentions here shown only for
the word ‘making’. Different colors represent different heads. Best viewed in color.
13
